<a href="https://colab.research.google.com/github/akshithaa1/chat_with_websites/blob/main/Chat_with_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 50.0 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to fetch and scrape content from a website
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract textual content from paragraphs and headers
    text_content = ""
    for p in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        text_content += p.get_text() + "\n"

    return text_content

# Example websites
urls = [
    "https://www.uchicago.edu/",
    "https://www.washington.edu/",
    "https://www.stanford.edu/",
    "https://und.edu/"
]

# Scrape content from each website
website_texts = {url: scrape_website(url) for url in urls}

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the embedding model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Convert the website texts into chunks (optional: segment into smaller chunks for better granularity)
def chunk_text(text, chunk_size=512):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Convert the scraped texts into chunks and then embeddings
embeddings = {}
for url, text in website_texts.items():
    chunks = chunk_text(text)
    embeddings[url] = model.encode(chunks, convert_to_tensor=True)

# Example: embedding of the first chunk from the first website
print(embeddings['https://www.uchicago.edu/'][0])


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tensor([-1.1127e-01,  2.6041e-01,  6.9312e-02, -9.3087e-02,  1.6532e-01,
        -4.7732e-01, -2.3044e-01, -3.0352e-02, -5.0542e-02, -6.4156e-02,
         1.5811e-04, -1.9026e-01, -9.8942e-04, -2.3931e-01, -1.0899e-01,
         1.8687e-01, -1.1690e-01, -2.3213e-01, -2.2616e-02, -1.1849e-01,
        -5.9335e-01, -4.8225e-01, -8.0040e-01, -1.2788e-01,  3.3482e-02,
         2.1483e-01,  3.9024e-01,  1.4654e-01, -4.2779e-01,  4.1327e-01,
         2.5291e-01,  4.5349e-01,  6.4012e-01,  1.1965e-01,  2.8337e-01,
         8.7219e-02, -1.4230e-01,  2.1324e-01,  1.0979e-01, -2.0712e-01,
        -9.8423e-02, -2.7995e-01,  1.5563e-01, -4.8516e-02,  3.5663e-01,
        -2.1792e-02, -1.6736e-01,  2.6265e-01,  2.4169e-01, -3.8690e-02,
         8.6932e-02, -5.2786e-03,  5.2582e-03, -1.1794e-01, -1.4769e-01,
        -1.4597e-01,  1.3517e-01,  2.0625e-01, -9.3040e-02, -2.8140e-01,
         3.8690e-01, -4.9403e-01, -4.6636e-01, -3.6107e-01,  1.1330e-01,
        -3.8848e-02,  2.4834e-01, -1.5364e-01, -1.6

In [ ]:
import faiss

# Function to store embeddings in FAISS
def store_embeddings_in_faiss(embeddings_dict):
    index = faiss.IndexFlatL2(embeddings_dict['https://www.uchicago.edu/'][0].shape[0])  # Dimension from one of the embeddings

    for url, emb_list in embeddings_dict.items():
        for emb in emb_list:
            emb_np = emb.cpu().numpy().reshape(1, -1)  # Convert to numpy array and reshape
            index.add(emb_np)

    return index

# Store embeddings in FAISS
index = store_embeddings_in_faiss(embeddings)

In [ ]:
generator = pipeline('text-generation', model='distilgpt2')

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
!pip install huggingface_hub
from huggingface_hub import login
login()  # This will prompt you to enter your Hugging Face credentials

In [ ]:
from transformers import pipeline

# Use a publicly available model like 'distilgpt2' to avoid authentication issues
generator = pipeline('text-generation', model='distilgpt2')

# Define the generate_response function before calling it
def generate_response(query, index, embeddings_dict):
    query_embedding = model.encode([query], convert_to_tensor=True)  # Embed the query
    query_np = query_embedding.cpu().numpy().reshape(1, -1)  # Convert to numpy array

    # Perform the similarity search in FAISS

In [ ]:
from transformers import pipeline
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Initialize the generator using distilgpt2
generator = pipeline('text-generation', model='distilgpt2')

# Initialize the embedding model (assuming you are using SentenceTransformers)
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')  # Use a pre-trained model for embeddings

# Function to generate a response using the retrieved context from FAISS
def generate_response(query, index, embeddings_dict, top_k=3):
    # Embed the query
    query_embedding = sentence_model.encode([query], convert_to_tensor=True)
    query_np = query_embedding.reshape(1, -1)  # Convert to numpy array

    # Perform the similarity search in FAISS
    D, I = index.search(query_np, top_k)  # Search for top_k similar embeddings

    # Retrieve the relevant text chunks based on the search results
    similar_texts = []
    for idx in I[0]:
        # Safely check if the index exists in embeddings_dict
        if idx in embeddings_dict:
            similar_texts.append(embeddings_dict[idx])
        else:
            similar_texts.append(f"Text not found for index {idx}")  # Fallback text if index is not found

    # Create the prompt by combining the query and the similar texts
    context = "\n".join(similar_texts)  # Join the texts into a single context string
    prompt = f"Query: {query}\nContext:\n{context}\nAnswer:"

    # Use the LLM to generate a response
    response = generator(prompt, max_new_tokens=100, num_return_sequences=1)

    return response

# Example query
query = "What is the history of the University of Chicago?"

# Assuming you have an index (FAISS) and embeddings_dict already created
# Generate the response using the query and the stored embeddings in FAISS
response = generate_response(query, index, embeddings)
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'Query: What is the history of the University of Chicago?\nContext:\nText not found for index 1\nText not found for index 2\nText not found for index 4\nAnswer: "So much of the history of the University of Chicago, including all the other students who were in the classroom, was done to the university that the professors were working on."\nOn January 23, 2011, the university issued a press release saying, "We are honored to host this prestigious event in Chicago. The school will be participating in the second annual celebration of the campus\u200fs history," adding, "This event is designed to bring the power of historical data to inform the public of the'}]
